In [2]:
import pandas as pd
data = pd.read_csv("candy-data.csv", delimiter=',', index_col='competitorname')

In [3]:
#обучение модели будем проводить на данных, за исключением некоторых конфет
train_data = data.drop(['Payday','Rolo','Sixlets'])
train_data

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent,Y
competitorname,,,,,,,,,,,,,
100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725,1
3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936,1
One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086,0
One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505,0
Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Snickers Crisper,1,0,1,1,0,1,0,1,0,0.604,0.651,59.529251,1
Sour Patch Kids,0,1,0,0,0,0,0,0,1,0.069,0.116,59.863998,1
Sour Patch Tricksters,0,1,0,0,0,0,0,0,1,0.069,0.116,52.825947,1


In [4]:
#отбор данных для предикторов, удаление дввух последних столбцов, индекс не включается в данные автоматически.
X = train_data.drop(['winpercent', 'Y'], axis=1)

In [5]:
#отбор столбца для отклика
y = train_data['Y']

In [6]:
#подключение модели логистической регрессии из библиотеки sklearn
from sklearn.linear_model import LogisticRegression

In [7]:
#обучение модели
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel())

In [8]:
#предсказание для сладости из таблицы
#выбор строки из таблицы
Grand = train_data.loc['100 Grand',:].to_frame().T
#отбор данных для предикторов и предсказание
reg.predict(Grand.drop(['winpercent', 'Y'], axis=1))

array([1])

In [9]:
#предсказание для сладости, введенной вручную, вероятности указаны для классов 0 и 1 соответственно 
reg.predict_proba([[1, 1, 1, 0, 0, 1, 1, 1, 0, 0.64, 0.76]])

array([[0.16802128, 0.83197872]])

In [244]:
test_data = pd.read_csv("candy-test.csv", delimiter=',', index_col='competitorname')


In [245]:
X_test = pd.DataFrame(test_data.drop(['Y'], axis=1))

#предсказание с помощью обученной модели, порог отсечения по умолчанию составляет 0.5
Y_pred = reg.predict(X_test)

In [246]:
#отбор отклика Y из тестовых данных и преобразование в массив
Y_true = (test_data['Y'].to_frame().T).values.ravel()
print(Y_true)

[0 0 0 1 1 0 0 1 0 1 0 0 0 0 1]


In [247]:
#предсказание для сладости из таблицы
#выбор строки из таблицы
trolls = test_data.loc['Trolli Sour Bites',:].to_frame().T
trolls = trolls.drop(['Y'], axis=1)
trolls

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent
Trolli Sour Bites,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.313,0.255


In [248]:
reg.predict(trolls)

array([0])

In [249]:
#предсказание для сладости из таблицы
#выбор строки из таблицы
daddies = test_data.loc['Sugar Daddy',:].to_frame().T
#отбор данных для предикторов и предсказание
reg.predict(daddies.drop(['Y'], axis=1))

array([0])

In [250]:
#подключение библиотеки для вычисления метрик
from sklearn import metrics

In [251]:
#вычисление FPR и TPR, с последующим нахождением AUC
fpr, tpr, thresholds = metrics.roc_curve(Y_true, Y_pred)
metrics.auc(fpr, tpr)

0.6499999999999999

In [252]:
#вычисление Recall
metrics.recall_score(Y_true, Y_pred)

0.6

In [253]:
#вычисление Precision
metrics.precision_score(Y_true, Y_pred)

0.5